In [1]:
!pip install -r requirements.txt

  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached pyarrow-18.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.1.0+cu118 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.1.0+cu118

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install datasets
!pip install transformers
!pip install --upgrade transformers
!pip install einops
!pip install trl
!pip install huggingface_hub
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Found existing installation: unsloth 2025.1.5
Uninstalling unsloth-2025.1.5:
  Successfully uninstalled unsloth-2025.1.5
  Cloning https://

### Loading in model from Unsloth

In [3]:
import torch
from unsloth import FastLanguageModel

# Clearing GPU memory cache
torch.cuda.empty_cache()

# Set model device type to 'cuda'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Using unsloth fast library
model, tokenizer =  FastLanguageModel.from_pretrained("unsloth/Llama-3.2-3B",
                                                      max_seq_length=2048,
                                                      dtype=None, 
                                                      load_in_4bit=True )

tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = "Alpaca"

# Using LoRA adapters to only update part of the model's parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",   
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
Using device: cuda:0
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Dataset for training

In [4]:
alpaca_prompt = """
{}

{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    outputs      = examples["label"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
train_dataset = load_dataset("gz25/JGTV_SFT_Training", split = "train")
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)

## SFT Training

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        output_dir="Llama3.2_3B-SFT",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps=60,
        learning_rate = 2e-4,
        save_strategy = "steps",
        save_steps = 50,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
    )

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    max_seq_length=2048,
    train_dataset=train_dataset,
    dataset_text_field="text",
    dataset_num_proc=2,
    packing=False, # Can make training 5x faster for short sequences.
    args=training_args,
)

print("Starting training...")

trainer.train()

# Save trained model
save_dir = "Llama3.2_3B_SFTtrained"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Trained model has been saved to {save_dir}")

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,052 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 768
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,2.591800
2,2.506200
3,2.560700
4,2.678000
5,2.532300
6,2.258900
7,2.204600
8,2.004700
9,1.861900
10,1.943600


Trained model has been saved to Llama3.2_3B_SFTtrained


## Testing after training

In [2]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-9wfguda3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-9wfguda3
  Resolved https://github.com/huggingface/transformers.git to commit 5fa35344755d8d9c29610b57d175efd03776ae9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Set model device to 'cuda'
torch.set_default_device("cuda")

# Initialize model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Llama3.2_3B_SFTtrained")
model = AutoModelForCausalLM.from_pretrained("./Llama3.2_3B_SFTtrained")

tokenizer.chat_template = "Alpaca"


/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [6]:
import numpy as np

prompt = "There is most likely going to be fog around\n A. a marsh B. a tundra C. the plains D. a desert\n Respond and give your confidence in percentage terms"

# prompt = "There is most likely going to be fog around\n A. a marsh B. a tundra C. the plains D. a desert\nPlease answer the question to the best of your ability with the corresponding letter representing your answer and the whole number percentage representing your confidence in that answer."

inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to("cuda")

output = model.generate(**inputs, max_new_tokens=150, 
                         do_sample=False, 
                         temperature=1.0, 
                         output_logits=True,
                         return_dict_in_generate=True)

logits = output.logits

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]

generated_tokens = output.sequences[:, input_length:]

# Apply softmax function to logits tensors
log_probs = torch.stack(logits, dim=1).softmax(-1)

# Extract probability that corresponds with generated tokens based on the softmax output
gen_probs = torch.gather(log_probs, 2, generated_tokens[:, :, None]).squeeze(-1)

token_probs = {}

for tok, score in zip(generated_tokens[0], gen_probs[0]):
    # Format token string | probability
    # print(f" {tokenizer.decode(tok)} | {score.cpu().numpy()}")
    token_probs[tokenizer.decode(tok)] = score.cpu().numpy() * 100
    
output_answer = tokenizer.batch_decode(output.sequences, skip_special_tokens=True)[0]

print(output_answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


There is most likely going to be fog around
 A. a marsh B. a tundra C. the plains D. a desert
 Respond and give your confidence in percentage terms with that answer.

Answer: A
Confidence: 100% Percentage Who Answered: 100%
